In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

def string_to_bin(message):
    bin_ = []
    for c in message:
        bin_.append('{0:08b}'.format(ord(c)))
    return bin_

def block_int_to_bin(block):
    arr_bin = ['{0:08b}'.format(int(e)) for e in block]
    np_bin = []
    for a in arr_bin:
        r = [int(b) for b in a]
        np_bin.append(r)
    return np.array(np_bin)

def block_bin_to_int(block):
    return block

def hide_in_block(block, word):
    key = np.array([
            [1,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,1,0,0,0,1,0],
            [0,0,0,0,1,0,0,0],
            [0,0,0,0,0,0,1,0],
            [0,1,0,1,0,1,0,0]
        ])
    bin_word = '{0:08b}'.format(ord(word))
    print('Block:\n',block,'\nSecret word:',word,'====>',bin_word)
    for i in range(0,8):
        fi = block_int_to_bin(block[:,i])
        # SUM(fi and key)
        and_sum = np.bitwise_and(fi, key).sum()
        # SUM(key)
        key_sum = key.sum()
        print('\nSUM(fi and key):',and_sum,'| SUM(key):',key_sum,'| Bit:',bin_word[i])
        if and_sum % 2 == int(bin_word[i]):
            print('====> Do nothing')
        elif and_sum == 1:
            print('====> and_sum == 1')
        elif and_sum == key_sum - 1:
            print('====> and_sum == key_sum - 1')
        else:
            print('====> Everything else')

hide_in_block(
    np.array(
        [[143, 145, 147, 146, 147, 147, 148, 148],
        [143, 144, 145, 145, 144, 145, 145, 146],
        [144, 145, 146, 145, 144, 144, 146, 146],
        [144, 144, 143, 145, 145, 146, 144, 146],
        [144, 143, 145, 145, 145, 144, 145, 145],
        [146, 145, 144, 144, 145, 148, 147, 146],
        [147, 146, 145, 144, 145, 146, 148, 147],
        [146, 146, 143, 145, 145, 146, 147, 147]]
    ),
    'd'
)


Block:
 [[143 145 147 146 147 147 148 148]
 [143 144 145 145 144 145 145 146]
 [144 145 146 145 144 144 146 146]
 [144 144 143 145 145 146 144 146]
 [144 143 145 145 145 144 145 145]
 [146 145 144 144 145 148 147 146]
 [147 146 145 144 145 146 148 147]
 [146 146 143 145 145 146 147 147]] 
Secret word: d ====> 01100100

SUM(fi and key): 4 | SUM(key): 12 | Bit: 0
====> Do nothing

SUM(fi and key): 4 | SUM(key): 12 | Bit: 1
====> Everything else

SUM(fi and key): 3 | SUM(key): 12 | Bit: 1
====> Do nothing

SUM(fi and key): 2 | SUM(key): 12 | Bit: 0
====> Do nothing

SUM(fi and key): 2 | SUM(key): 12 | Bit: 0
====> Do nothing

SUM(fi and key): 3 | SUM(key): 12 | Bit: 1
====> Do nothing

SUM(fi and key): 2 | SUM(key): 12 | Bit: 0
====> Do nothing

SUM(fi and key): 3 | SUM(key): 12 | Bit: 0
====> Everything else
